# Język Python - Wykład 7. Metaclass

(Opracowanie oparte na świetnym wpisie StackOverflow @e-satis: http://stackoverflow.com/questions/100003/what-is-a-metaclass-in-python)

Tim Peters (ten od PEP 20 - Zen of Python oraz od timsort!)


<img src="files/L7_img/tim.jpg">


powiedział:

***Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don't (the people who actually need them know with certainty that they need them, and don't need an explanation about why).***

## Klasy jako obiekty

Definicje klas służą nam zwykle jako kod, za pomocą którego tworzone są obiekty:


In [ ]:
class ObjectCreator(object):
    pass

In [ ]:
print(ObjectCreator())

Ale klasy są także same w sobie... obiektami!

In [ ]:
print(ObjectCreator)

Oznacza to, że obiekt reprezentujący klasę ma umiejętność samemu tworzenia (instancjonowania) innych obiektów. Z tego powodu jest właśnie klasą!

Ale skoro każda klasa jest obiektem to oczywiście możemy:
 
 * przypisać ją do zmiennej,
 * skopiować ją,
 * dodać do niej atrybut,
 * przekazać ją jako argument funkcji.


In [ ]:
FooCreator = ObjectCreator
print(FooCreator())

In [ ]:
print(hasattr(ObjectCreator, 'bar'))

In [ ]:
ObjectCreator.bar = 'baz'

In [ ]:
print(hasattr(ObjectCreator, 'bar'))

In [ ]:
def echo(x):
    print(x)

echo(ObjectCreator)

## Dynamiczne tworzenie klas

Ponieważ klasy są obiektami, można je tworzyć dynamicznie (w run-time):

In [ ]:
def choose_class(name):
    if name == 'foo':
        class Foo(object):
            pass
        return Foo # return the class, not an instance
    else:
        class Bar(object):
            pass
        return Bar

In [ ]:
print(choose_class('foo'))
print(choose_class('bar'))

In [ ]:
print(choose_class('bar')())

To podejście nie jest jednak w pełni dynamiczne, bo nadal gdzieś jest explicite napisany kod definiujący tą klasę.

Ponieważ klasy są obiektami, to także i one muszą być przez coś tworzone. Gdy piszemy deklarację klasy i używamy słowa kluczowego **class** interpreter Pythona wywołuje pewny kod, który tworzy tę klasę i zapisuje ją pod podaną nazwą w namespace (podobnie jak dekorator, wywołuje kod i zwraca obiekt callable do podstawienia pod daną nazwą).

W Pythonie tworzenie obiektu klasy odbywa się explicite poprzez użycie funkcji type.

In [ ]:
print(type(1))

In [ ]:
print(type("1"))

In [ ]:
print(type(ObjectCreator()))

In [ ]:
print(type(ObjectCreator))

In [ ]:
class MyShinyClass(object):
    pass
print(MyShinyClass)

In [ ]:
print(type('MyShinyClass', (), {}))

In [ ]:
ms = type('MyShinyClass', (), {})
print(ms())

In [ ]:
class Foo(object):
    bar = True

class FooChild(Foo):
    baz = False

In [ ]:
FooChild = type('FooChild', (Foo,), {'baz': False})
ob = FooChild()
print(ob.bar, ob.baz)

## Czym są metaklasy?

**Metaklasa jest tym czymś co tworzy obiekt reprezentujący klasę w Pythonie.**

Można to sobie wyobrazić w ten sposób:

```
MyClass = MetaClass()
MyObject = MyClass()
```

Prównajmy to z tym co już wiemy o tworzeniu klas:

```
MyClass = type('MyClass', (), {})
```

Z tego wynika, że samo **type** jest metaklasą w Pythonie. Co więcej jest ono domyślną metaklasą używaną zawsze standardowo, gdy Python stara się utworzyć jakąś klasę z jej deklaracji i definicji.

**Dlaczego type() jest przeciążone?**

Ze względu zaszłości językowych i kompatybilności wstecznej

**Dlaczego type() jest pisane z małej litery skoro to metaklasa?**

Ze względu na konwencję innych typów, int, str, unicode (to było pisane z małej litery w Pythonie).

In [ ]:
age = 35
print(age.__class__)

In [ ]:
name = 'bob'
print(name.__class__)

In [ ]:
def foo(): pass
print(foo.__class__)

In [ ]:
print(ObjectCreator().__class__)

W takim razie - jaka jest klasa obiektu reprezentująca klasę klasy? 

In [ ]:
print(ObjectCreator.__class__.__class__)

In [ ]:
print(age.__class__.__class__)

In [ ]:
print(name.__class__.__class__)

A więc podsumowując - metaklasa to nic innego jak "fabryka klas". **type** jest standardowa metaklasą w Pythonie, ale Python pozwala nam ją podmienić jeśli chcemy!

## ``__metaclass__``

Za pomocą specjalnego atrybutu klasy  ``__metaclass__`` można wskazać jakiej metaklasy ma użyć Python do tworzenia tej klasy.

```
class Foo(object):
    __metaclass__ = some_metaclass
    ...
```

Uwaga - jest to pewien hack języka. W momencie deklaracji klasy **class Foo(object)** tak naprawdę obiekt tej klasy nie jest jeszcze tworzony. Najpierw przeszukiwana jest jej definicja czy znajduje się tam **``__metaclass__``** - jeśli tak, jest dopiero wtedy używana do utworzenia obiektu/klasy Foo. Jeśli nie ma - użyty zostanie **type**.

A co z dziedziczeniem?

```
class Bar(Foo):
    pass
```

Python sprawdzi:

 * Czy Bar zawiera atrybut ``__metaclass__``? 
 * Czy dowolny rodzic klasy (np. Foo) zawiera atrybut ``__metaclass__``? 
 * Czy moduł zawiera atrybut ``__metaclass__``?
 * Jeśli żadne z powyższych się nie powiodło - użyje **type**.

## Tworzenie własnej metaklasy

Metaklas możemy potrzebować, jeśli chcemy modyfikować obiekty reprezentujące klasy podczas ich tworzenia. Jest to szczególnie pomocne przy tworzeniu API. Django ORM (modele) zostały utworzone właśnie z użyciem metaklas.

Dla uproszczenia przyjmijmy, że chcemy utworzyć metaklasę, która zapewni, że wszystkie atrybuty klasy będą miały w nazwie tylko WIELKIE LITERY, niezależnie od tego jak programista napiszę definicję klasy.

**Metaklasą** (pomimo nazwy) może być cokolwiek co da się wywołać (jest callable) i przyjmie argumenty takie same jak **type(name, bases, dict)**

In [ ]:
# the metaclass will automatically get passed the same argument
# that you usually pass to `type`
def upper_attr(future_class_name, future_class_parents, future_class_attr):
  """
    Return a class object, with the list of its attribute turned 
    into uppercase.
  """

  # pick up any attribute that doesn't start with '__' and uppercase it
  uppercase_attr = {}
  for name, val in future_class_attr.items():
      if not name.startswith('__'):
          uppercase_attr[name.upper()] = val
      else:
          uppercase_attr[name] = val

  # let `type` do the class creation
  return type(future_class_name, future_class_parents, uppercase_attr)

In [ ]:
class Foo(object):
    __metaclass__ = upper_attr
    
    foo = 1
    bar = 2

In [ ]:
Foo.FOO

In [ ]:
Foo.BAR

In [ ]:
hasattr(Foo, 'bar')

In [ ]:
dir(Foo())

Metaklasa nie nazywała by się meta**klasą** jeśli tego samego nie można by było osiągnąć używając **klasy** zamiast **funkcji**.

In [ ]:
# remember that `type` is actually a class like `str` and `int`
# so you can inherit from it
class UpperAttrMetaclass(type): 
    # __new__ is the method called before __init__
    # it's the method that creates the object and returns it
    # while __init__ just initializes the object passed as parameter
    # you rarely use __new__, except when you want to control how the object
    # is created.
    # here the created object is the class, and we want to customize it
    # so we override __new__
    # you can do some stuff in __init__ too if you wish
    # some advanced use involves overriding __call__ as well, but we won't
    # see this
    def __new__(upperattr_metaclass, future_class_name, 
                future_class_parents, future_class_attr):

        uppercase_attr = {}
        for name, val in future_class_attr.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        return type.__new__(future_class_name, future_class_parents, uppercase_attr)

In [ ]:
class UpperAttrMetaclass(type): 

    def __new__(cls, clsname, bases, dct):

        uppercase_attr = {}
        for name, val in dct.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        return super(UpperAttrMetaclass, cls).__new__(cls, 
                clsname, bases, uppercase_attr)

"That's it"


<img src="files/L7_img/okay-thats-it-everyone-out-of-the-gene-pool.jpg"> 

## Metaklasy - raz jeszcze, krócej

1. Przechwytujemy akt tworzenia klasy
2. Modyfikujemy nowo tworzoną klasę
3. Zwracamy (podstawiamy) 

## Zastosowanie metaklas

Metaklasy w Pytonie to czarna magia, jeśli pytasz po co masz to stosować, to oznacza, że napewno nie powinieneś tego stosować. Przypomnijmy raz jeszcze co powiedział Tim Peters:

*Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don't (the people who actually need them know with certainty that they need them, and don't need an explanation about why).*

Jedynym sensownym użyciem metaklas jest tworzenie API. Raz jeszcze o django ORM:

```
class Book(models.Model):
    name = models.CharField(max_length=50)
    pages = models.IntegerField()

django_book = Book(name='The Django', pages='333')
>>> print django_book.pages
333
```

Zwykle nie ma sensu używać metaklas. Są zbyt skomplikowane. Klasy możemy spokojnie modyfikować z użyciem:

 * dekoratorów klas
 * monkey patchingu

## Materiały wideo



<iframe
                width="640"
                height="480"
                src="http://www.youtube.com/embed/j82bS2jzS2I"
                frameborder="0"
                allowfullscreen
            ></iframe>